# My Playground for Neural Network

In [ ]:
from utils import *
from stratified import *
from run import *
from neuralnetwork import *

In [ ]:
def importhousedata():
    house = importfile('hw3_house_votes_84.csv', ',')
    housecategory = {}
    for i in house[0]:
        housecategory[i] = 'categorical'
    housecategory["class"] = 'class'
    housedata = np.array(house[1:]).astype(float)
    return housedata, housecategory
def importwinedata():
    wine = importfile('hw3_wine.csv', '\t')
    winecategory = {}
    for i in wine[0]:
        winecategory[i] = 'numerical'
    winecategory["# class"] = 'class'
    winedata = np.array(wine[1:]).astype(float)
    return winedata, winecategory
def importcmcdata():
    cmc = importfile('cmc.data', ',')
    cmccategory = {"Wife's age":"numerical","Wife's education":"categorical",
    "Husband's education":"categorical","Number of children ever born":"numerical",
    "Wife's religion":"binary","Wife's now working?":"binary",
    "Husband's occupation":"categorical","Standard-of-living index":"categorical",
    "Media exposure":"binary","Contraceptive method used":"class"}
    cmcdata = np.array(cmc).astype(int)
    return cmcdata, cmccategory
def importcancerdata():
    cancer = importfile('hw3_cancer.csv', '\t')
    cancercategory = {}
    for i in cancer[0]:
        cancercategory[i] = 'numerical'
    cancercategory["Class"] = 'class'
    cancerdata = np.array(cancer[1:]).astype(float)
    return cancerdata, cancercategory

housedata, housecategory = importhousedata()
winedata, winecategory = importwinedata()
cmcdata,cmccategory = importcmcdata()
cancerdata, cancercategory = importcancerdata()
example_category = {'ece': 'numerical','cs': 'categorical','phy': 'categorical','math': 'categorical','musi': 'categorical','grade': 'class'}
#example_data = np.array([[100,0,1,1,0,0],[123,1,0,1,1,1],[44,1,0,0,0,2],[92,1,0,1,0,3],[73,0,1,0,0,2]])
example_data = np.array([[10,0,1,1,0,0],[23,1,0,1,1,1],[44,1,0,0,0,2],[92,1,0,1,0,3],[73,0,1,0,0,2],[122,0,1,1,0,0],[124,1,0,1,1,1],[44,1,0,0,0,2],[40,1,0,1,0,3],[73,0,1,0,0,2]])

In [ ]:
ohe_house_data, ohe_house_category = onehotencoder(housedata, housecategory)
ohe_wine_data, ohe_wine_category = onehotencoder(winedata,winecategory)
ohe_cmc_data, ohe_cmc_category = onehotencoder(cmcdata,cmccategory)
ohe_example_data, ohe_example_category = onehotencoder(example_data,example_category)
ohe_cancer_data, ohe_cancer_category = onehotencoder(cancerdata,cancercategory)

In [ ]:
n_ohe_house_data = normalizetrain(ohe_house_data,ohe_house_category)[0]
n_ohe_wine_data = normalizetrain(ohe_wine_data,ohe_wine_category)[0]
n_ohe_cmc_data = normalizetrain(ohe_cmc_data,ohe_cmc_category)[0]
n_ohe_example_data = normalizetrain(ohe_example_data,ohe_example_category)[0]
n_ohe_cancer_data = normalizetrain(ohe_cancer_data,ohe_cancer_category)[0]

In [ ]:
def g(x): # sigmoid function
    return np.exp(x) / (1 + np.exp(x))

In [ ]:
def transposelistoflist(l):
    newlistoflist = []
    for i in range(len(l[0])):
        newlist = []
        for j in range(len(l)):
            newlist.append(l[j][i])
        newlistoflist.append(newlist)
    return newlistoflist

In [ ]:
def gradientD(weights_list,deltalist,attributelist,biasterm=True):
    gradlist = []
    for i in range(len(weights_list)):
        attributenow = attributelist[i]
        deltanow = np.array([deltalist[i]]).T
        dotproduct = deltanow*attributenow
        # print('dotshape',dotproduct.shape)
        gradlist.append(dotproduct)
    return gradlist

In [ ]:
# Forward propagation vectorized
def neural_network(normed_hotted_data,ohe_category,weights_list,biasterm=True, minibatchk = 15, lambda_reg = 0.05, learning_rate = 0.0001):
    normed_ohe_copy = normed_hotted_data.copy()
    np.random.shuffle(normed_ohe_copy)
    splitted = np.array_split(normed_ohe_copy, minibatchk)
    
    inputcategory, outputcategory = [],[]
    inputindex, outputindex = [],[]
    n = 0
    for i in ohe_category:
        if ohe_category[i] != 'class_numerical':
            inputcategory.append(i) # name of the input category
            inputindex.append(n) # index of the input category
        else:
            outputcategory.append(i) # name of the output category
            outputindex.append(n) # index of the output category
        n += 1
    
    b = 1 if biasterm else 0
    
    for onebatch in splitted:
        onebatch = onebatch.T
        input_data = onebatch[inputindex].T
        output_data = onebatch[outputindex].T
        # input_data_mean = onebatch[inputindex].mean(axis=1)
        output_data = onebatch[outputindex].T

        # forward propagation
        instance_index = 0
        j = 0
        listofgradient = []
        for one_instance in input_data:
            current_layer_a = np.append(1,one_instance) if b == 1 else one_instance
            # input layer is the current layer
            current_layer_index = 0
            output_real = output_data[instance_index]
            attributesnobias = [one_instance]
            attributeswbias = [current_layer_a]
            for theta in weights_list:
                z = np.dot(theta,current_layer_a)
                a = g(z)
                current_layer_a = np.append(1,a) if (b == 1) and (current_layer_index+1 != len(weights_list)) else a
                attributesnobias.append(a)
                attributeswbias.append(current_layer_a)
                current_layer_index += 1

            output_predict = current_layer_a # the last attribute is the output for this batch.
            instance_index += 1
            j += costfunction(output_real,output_predict)

            # calculate delta blame (back propagation)
            listofdelta = blame(output_real,output_predict,weights_list, attributeswbias)
            thisgradient = gradientD(weights_list,listofdelta,attributeswbias,biasterm)
            listofgradient.append(thisgradient)
        
        gradientP = [lambda_reg*t for t in weights_list]
        grad_D_transpose = transposelistoflist(listofgradient)
        grad_D_sum = [np.sum(t,axis=0) for t in grad_D_transpose]
        gradients_batch = []
        for i in range(len(grad_D_sum)):
            gradients_batch.append((grad_D_sum[i] + gradientP[i])*(1/instance_index))
        
        j /= (instance_index+1)
        s = sumofweights(weights_list,bias=b)*lambda_reg/(2*(instance_index+1))
        allj = j+s # total cose with regularization

        # update weights
        for i in range(len(weights_list)):
            weights_list[i] -= learning_rate*gradients_batch[i]

    return weights_list, allj, j

In [ ]:
layerdataparameter = [4,5,4,4,3,5] # [First layer, Second layer, Third layer], not including the bias and input & output layer
initweights = initialize_weights(ohe_cmc_category,layerdataparameter)
kk,jj,pj = neural_network(n_ohe_cmc_data,ohe_cmc_category,initweights)

In [620]:
j_previous = 99
epsilon = 80
weight, j_now, purej = neural_network(n_ohe_cmc_data,ohe_cmc_category,initweights)
epsilon = j_previous - j_now
j_previous = j_now

while epsilon > 0.00001:
    weight, j_now, purej = neural_network(n_ohe_cmc_data,ohe_cmc_category,weight)
    print('purej',purej)
    print('j now is',j_now)
    epsilon = abs(j_previous - j_now)
    print(epsilon)
    j_previous = j_now

purej 2.333433598970095
j now is 2.3463690831363464
0.09037175651839702
purej 2.3495729896988258
j now is 2.362507901014347
0.016138817878000644
purej 2.3647016727676116
j now is 2.377636026274969
0.015128125260622127
purej 2.308761505593757
j now is 2.3216952987092876
0.05594072756568158
purej 2.4204855765596753
j now is 2.433418835039218
0.11172353632993026
purej 2.2673279429460327
j now is 2.280260658216
0.15315817682321775
purej 2.3217054271482023
j now is 2.3346376144916645
0.05437695627566441
purej 2.3171318959119214
j now is 2.3300635498719826
0.004574064619681906
purej 2.3160692761063375
j now is 2.3290004200357166
0.0010631298362659969
purej 2.2873016832253668
j now is 2.3002323123407296
0.028768107694987055
purej 2.2069214269682864
j now is 2.2198515414494553
0.08038077089127427
purej 2.31280545605288
j now is 2.3257350897794735
0.10588354833001823
purej 2.3621670553617204
j now is 2.375096192263004
0.04936110248353032
purej 2.235293976860665
j now is 2.248222608104036
0.1268

KeyboardInterrupt: 

In [ ]:
print(j_now)
correctweight = weight

1.8299048672859848


In [ ]:
# def predictusingnetwork(inputdata, category, weight):


IndentationError: expected an indented block after function definition on line 1 (4157862203.py, line 2)

In [ ]:
layernow = 3
layernext = 2
init_weight = np.random.rand(layernext,layernow) * 2 - 1
a_now = np.random.rand(layernow,1)
print(init_weight)
print(a_now)
c = np.dot(init_weight,a_now)
print(c)

[[ 0.55549085 -0.70042758  0.13038175]
 [ 0.72195971  0.91836836 -0.86859995]]
[[0.92042182]
 [0.99931617]
 [0.26456833]]
[[-0.15416782]
 [ 1.35244379]]
